Я возьму для рассмотрения stanza, pymorphy и natasha.

In [1]:
import re
import pymorphy2
import stanza
from sklearn.metrics import accuracy_score
from string import punctuation
from copy import deepcopy
from collections import Counter

morph = pymorphy2.MorphAnalyzer()
stanza.download('ru')
nlp = stanza.Pipeline('ru')

2021-10-03 12:01:13 INFO: Downloading default packages for language: ru (Russian)...


2021-10-03 12:01:16 INFO: File exists: C:\Users\User\stanza_resources\ru\default.zip.
2021-10-03 12:01:22 INFO: Finished downloading models and saved to C:\Users\User\stanza_resources.
2021-10-03 12:01:22 INFO: Loading these models for language: ru (Russian):
| Processor | Package   |
-------------------------
| tokenize  | syntagrus |
| pos       | syntagrus |
| lemma     | syntagrus |
| depparse  | syntagrus |
| ner       | wikiner   |

2021-10-03 12:01:22 INFO: Use device: cpu
2021-10-03 12:01:22 INFO: Loading: tokenize
2021-10-03 12:01:27 INFO: Loading: pos
2021-10-03 12:01:27 INFO: Loading: lemma
2021-10-03 12:01:28 INFO: Loading: depparse
2021-10-03 12:01:28 INFO: Loading: ner
2021-10-03 12:01:30 INFO: Done loading processors!


In [2]:
punctuation += "–«»…"
punctuation = punctuation.replace("_", "")
punctuation = punctuation.replace("-", "")

In [3]:
from natasha import (
    Segmenter,
    MorphVocab,
    
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    
    PER,

    Doc
)


segmenter = Segmenter()
morph_vocab = MorphVocab()

emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
ner_tagger = NewsNERTagger(emb)


В качестве корпуса я собрала несколько предложений из Лимонова, потому что у него специфичный стиль, много окказионализмов и иногда встречается что-то интересное по синтаксису.

In [260]:
with open('eto_ya_edichka.txt', encoding='UTF-8') as f:
    text = f.read()
    
text[:303] #сам корпус размечен примерно так

'Комнатка_NOUN моя_PRON_ADJ имеет_VERB 4_NUM шага_NOUN в_PREP длину_NOUN и_CONJ 3_NUM в_PREP ширину_NOUN. На_PREP стенах_NOUN, прикрывая_ADV_VERB пятна_NOUN, оставшиеся_ADJ_VERB от_PREP прежних_ADJ жильцов_NOUN, висят_VERB: большой_ADJ портрет_NOUN Мао_NOUN Цзэ_NOUN Дуна_NOUN; портрет_NOUN Патриции_NOUN'

In [220]:
text_to_show = re.sub("_[A-Z]{3,4}|\n", "", text)
text_to_show

'Комнатка моя имеет 4 шага в длину и 3 в ширину. На стенах, прикрывая пятна, оставшиеся от прежних жильцов, висят: большой портрет Мао Цзэ Дуна; портрет Патриции Хёрст; портрет Андре Бретона, основателя сюрреалистической школы, который я вожу с собой уже много лет, и которого Андре Бретона обычно никто их приходящих ко мне не знает; плакат, призывающий голосовать за Рабочей партии кандидатов; картины моего друга художника Хачатуряна. В изголовье кровати у меня плакат – «За Вашу и Нашу свободу», оставшийся от демонстрации у здания «Нью Йорк Таймз». Ну я был поэт, поэт я был, раз уж вам хочется знать кто, неофициальный поэт был, подпольный, было да сплыло, а теперь я один из ваших, я подонок, я тот кого вы кормите щами, кого вы поите дешёвым и дрянным калифорнийским вином – 3.59 галлоновая бутыль, а я все равно вас презираю.Остальной персонал отеля относится ко мне так-сяк.Я не знаю, какие там комнаты, но место там похуже, куда более блатное.Он перебивается с хлеба на воду, худ как скеле

Тут много имён собственных, разговорных слов («так-сяк»), слов, которые могут выступать как несколько частей речи («как» союз и частица, «правда» существительное и частица), есть редкая форма «худ», всякие явно несловарные слова («галлоновый», «лексингтоновский»), и слова, написанные через дефис («кубинец-швейцар» и «светло-зелёный»).

In [227]:
text_wo_punct = text.translate(str.maketrans('', '', punctuation))
text_wo_punct = re.sub('\n', ' ', text_wo_punct)

In [228]:
gold_POS = []
text_rem = text_wo_punct
while text_rem:
    tag_pos = re.search("(_[A-Z]{3,4}){1,2}", text_rem)
    word = text_rem[:tag_pos.start(0)]
    gold_POS.append([word, tag_pos.group(0)[1:]])
    text_rem = text_rem[tag_pos.end(0):].lstrip()
    
for i in range(len(gold_POS)):
    gold_POS[i][1] = gold_POS[i][1].split('_')
    
gold_POS[:10]

[['Комнатка', ['NOUN']],
 ['моя', ['PRON', 'ADJ']],
 ['имеет', ['VERB']],
 ['4', ['NUM']],
 ['шага', ['NOUN']],
 ['в', ['PREP']],
 ['длину', ['NOUN']],
 ['и', ['CONJ']],
 ['3', ['NUM']],
 ['в', ['PREP']]]

In [230]:
text_wo_punct = re.sub("_[A-Z]{3,4}", "", text_wo_punct)

## Stanza

In [224]:
stanza_doc = nlp(text_wo_punct)

In [225]:
stanza_POS = []
for sent in stanza_doc.sentences:
    for word in sent.words:
        stanza_POS.append([word.text, word.upos])
        
stanza_POS = [i for i in stanza_POS if i[0] not in punctuation+'-']

stanza_POS[:10]

[['Комнатка', 'NOUN'],
 ['моя', 'DET'],
 ['имеет', 'VERB'],
 ['4', 'NUM'],
 ['шага', 'NOUN'],
 ['в', 'ADP'],
 ['длину', 'NOUN'],
 ['и', 'CCONJ'],
 ['3', 'NUM'],
 ['в', 'ADP']]

## Natasha

In [231]:
natasha_doc = Doc(text_wo_punct)

natasha_doc.segment(segmenter)
natasha_doc.tag_morph(morph_tagger)
natasha_POS = []
for token in natasha_doc.tokens:
    natasha_POS.append([token.text, token.pos])
natasha_POS = [i for i in natasha_POS if i[1] != 'PUNCT']

natasha_POS[:10]

[['Комнатка', 'PROPN'],
 ['моя', 'DET'],
 ['имеет', 'VERB'],
 ['4', 'NUM'],
 ['шага', 'NOUN'],
 ['в', 'ADP'],
 ['длину', 'NOUN'],
 ['и', 'CCONJ'],
 ['3', 'NUM'],
 ['в', 'ADP']]

## Pymorphy2

In [232]:
pymor_POS = []
for word in text_wo_punct.split():
    pymor_POS.append([word, morph.parse(word.lower())[0].tag.POS])
    
pymor_POS[:10]

[['Комнатка', 'NOUN'],
 ['моя', 'ADJF'],
 ['имеет', 'VERB'],
 ['4', None],
 ['шага', 'NOUN'],
 ['в', 'PREP'],
 ['длину', 'NOUN'],
 ['и', 'CONJ'],
 ['3', None],
 ['в', 'PREP']]

# Сравнение

Для начала приведём теги к одному виду. Я размечала наиболее привычными для себя тегами, в основном они совпадают с pymorphy2, только у меня в стандарте нет разделения прилагательных и причастий на краткие/полные, разделения союзов и прочего. Местоимения имеют как свой собственный тег, так и тег той части речи, на которую они похожи (например, ADJ для «который»), потому что какой-то из теггеров так их размечал, а также двойные теги у деепричастий (наречие+глагол) и причастий (прилагательное+глагол), потому что, опять же, теггеры рассматривали их по-разному и не хотелось какую-то точку зрения обижать.

In [233]:
def tag_levelling(tagset):
    tags = [i[1] for i in tagset]
    for i in range(len(tags)):
        if str(tags[i]) == "PROPN":
            tags[i] = "NOUN"
        elif str(tags[i]) == "ADP":
            tags[i] = "PREP"
        elif str(tags[i]) in ["DET", "NPRO"]:
            tags[i] = "PRON"
        elif str(tags[i]) in ["CCONJ", "SCONJ"]:
            tags[i] = "CONJ"
        elif str(tags[i]) in ["AUX", "INFN", "GRND"]:
            tags[i] = "VERB"
        elif str(tags[i]) in ["ADJF", "ADJS", "PRTF", "PRTS"]:
            tags[i] = "ADJ"
        elif str(tags[i]) == "PRCL":
            tags[i] = "PART"
        elif str(tags[i]) == "ADVB":
            tags[i] = "ADV"
        elif str(tags[i]) == "NUMR":
            tags[i] = "NUM"
        else:
            tags[i] = str(tags[i])
        
    for i in range(len(tagset)):
        tagset[i][1] = tags[i]
    
    return tagset

In [234]:
stanza_POS = tag_levelling(stanza_POS)
natasha_POS = tag_levelling(natasha_POS)
pymor_POS = tag_levelling(pymor_POS)

Теперь, когда у нас все теги одного вида, можно их сравнивать.

In [235]:
def for_comparison(a):
    
    m = deepcopy(gold_POS)
    
    # m — manually tagged
    # a — automatically tagged
    
    auto_word = [i[0] for i in a]
    auto_tag = [i[1] for i in a]
    tags_to_compare = []
    for i in range(len(m)):
        word = m[i][0]
        if word == auto_word[0]:
            tags_to_compare.append(auto_tag[0])
            auto_word = auto_word[1:]
            auto_tag = auto_tag[1:]
        # на случай, если слово с дефисом разделено на два
        elif word == auto_word[0]+'-'+auto_word[1]:
            tags_to_compare.append(auto_tag[1])
            auto_word = auto_word[2:]
            auto_tag = auto_tag[2:]
            
        # если есть два варианта разбора слова, оставим тот, который совпадает с парсером, либо просто последний. 
        if len(m[i][1]) > 1:
            if m[i][1][0] == tags_to_compare[-1]:
                m[i][1] = [m[i][1][0]]
            else:
                m[i][1] = [m[i][1][1]]
                
        gold_tags = [i[1][0] for i in m]
        
            
    return tags_to_compare, gold_tags     

In [236]:
stanza_tags, gold_stanza_tags = for_comparison(stanza_POS)
natasha_tags, gold_natasha_tags = for_comparison(natasha_POS)
pymor_tags, gold_pymor_tags = for_comparison(pymor_POS)

print('STANZA\t\t', accuracy_score(gold_stanza_tags, stanza_tags))
print('NATASHA\t\t', accuracy_score(gold_natasha_tags, natasha_tags))
print('PYMORPHY2\t', accuracy_score(gold_pymor_tags, pymor_tags))

STANZA		 0.9427480916030534
NATASHA		 0.9007633587786259
PYMORPHY2	 0.9236641221374046


Лучший теггер — stanza. Соберём n-граммы:
1. прилагательное + существительное + существительное
2. наречие + прилагательное
3. "я" + глагол

In [255]:
adj_noun_noun = Counter()
adv_adj = Counter()
ya_verb = Counter()
for i in range(len(stanza_POS)-1):
    word, tag = stanza_POS[i][0], stanza_POS[i][1]
    next_word, next_tag = stanza_POS[i+1][0], stanza_POS[i+1][1]
    if i <= len(stanza_POS)-3:
        next_next_word, next_next_tag = stanza_POS[i+2][0], stanza_POS[i+2][1]
        if tag == "ADJ" and next_tag == 'NOUN' and next_next_tag == "NOUN":
            adj_noun_noun[word+" "+next_word+" "+next_next_word] += 1
    if tag == "ADV" and next_tag == "ADJ":
        adv_adj[word+" "+next_word] += 1
    elif word.lower() == "я" and next_tag == "VERB":
        ya_verb["я "+next_word] += 1

In [256]:
adj_noun_noun, adv_adj, ya_verb

(Counter({'большой портрет Мао': 1,
          'Рабочей партии кандидатов': 1,
          'Остальной персонал отеля': 1,
          'долговязого кубинца швейцара': 1,
          'солженицыновском Архипелаге Гулаг': 1}),
 Counter({'там похуже': 1, 'более блатное': 1}),
 Counter({'я вожу': 1, 'я был': 3, 'я глазею': 1}))

## Улучшение предыдущей домашки

К тем n-граммам, которые могли бы помочь в прошлый раз при определении тональности в отзывах на фильмы, можно отнести **не+глагол** (в отрицательных часто можно встретить «не зацепило», «не удалось», тем более «не понравилось»), **наречие+прилагательное** (скажем, условное «ужасно интересно», в котором при разделении точно обманчив окрас одного из элементов), и что-то типа **[фильм]+глагол+глагол/существительное/прилагательное(предикат)** — таких формулировок много при подведении итога («Фильм заставляет задуматься», «Картина вызывает отвращение», «„Оно“ получилось спорным» — правда, последняя n-грамма будет полезна только при сильно большом наборе данных.

Stanza неприлично долго обрабатывает тексты, поэтому собственно улучшения предыдущей программы не получилось :(